# 5월 IMC

## 임포트

In [2]:
import numpy as np # 넘파이
import matplotlib.pyplot as plt # 매트플롯립
import pandas as pd # 판다스(csv)
import re # 정규식 표현
from tqdm import tqdm

# 텐서플로우
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [6]:
# 불러오기
train = pd.read_csv('/Users/minguinho/Documents/AI_Datasets/IMC_May_data/IMC_May_train.csv', encoding='cp949')
test = pd.read_csv('/Users/minguinho/Documents/AI_Datasets/IMC_May_data/IMC_May_test.csv', encoding='cp949')

In [12]:
print("train : ", train.info())
print()
print("test : ", test.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Message_body  975 non-null    object
 1   Label         975 non-null    object
dtypes: object(2)
memory usage: 15.4+ KB
train :  None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Message_body  107 non-null    object
dtypes: object(1)
memory usage: 984.0+ bytes
test :  None
